# Download data from Mirbase

In [ ]:
base = "ftp://mirbase.org/pub/mirbase/CURRENT"
!wget {base}/miRNA.str.gz -P ./Data/          
!gzip -d ./Data/miRNA.str.gz 

# Convertor

In [1]:
import numpy as np
from tqdm import tqdm  # !pip install tqdm
from tqdm import trange

In [111]:
def isChar(c):
    return not c in [' ', '-', '|']

def adjust(text,n=7):
    text = str(text)    
    return " " * (n - len(text)) + text

def isMature(c1, c2):
    if(c1.isupper() and isChar(c1)):
        return True
    if(c2.isupper() and isChar(c2)):
        return True
    return False
    

def convertor(strText):
    # hairpin seq length
    number = 0 # inclue last
    for i in range(len(strText[2])):
        if(isChar(strText[2][i]) or isChar(strText[3][i])):
            number += 1    
        if(isChar(strText[4][i])):
            number += 1    
        if(isChar(strText[5][i]) or isChar(strText[6][i])):
            number += 1     
    # main loop
    f_counter = 1
    r_counter = 0
    index  = []
    values = []
    nucludid = []
    for i in range(len(strText[2])):
        ## check for '-'
        if(strText[6][i] == "-" and strText[5][i] == " " ):
            if(isChar(strText[2][i])):
                index.append(f_counter)
                values.append(0)            
                nucludid.append(strText[2][i])
                f_counter += 1      
                continue
            else:
                print('structure error!')
                continue
        if(strText[2][i] == "-" and strText[3][i] == " " ):
            if(isChar(strText[6][i])):
                index.append(number - r_counter)
                values.append(0)            
                nucludid.append(strText[6][i])
                r_counter += 1      
                continue
            else:
                print('structure error!')
                continue
        ##        
        if(isChar(strText[2][i]) and isChar(strText[6][i]) and 
            not isChar(strText[3][i]) and not isChar(strText[5][i]) and
            not isChar(strText[4][i])):        
            index.append(f_counter)
            values.append(0)        
            nucludid.append(strText[2][i])
        
            index.append(number - r_counter)
            values.append(0)             
            nucludid.append(strText[6][i])
            f_counter += 1        
            r_counter += 1                  
            continue
        
        if(not isChar(strText[2][i]) and not isChar(strText[6][i]) and 
             isChar(strText[3][i]) and  isChar(strText[5][i])):
            if(strText[4][i] == "|"):
                index.append(f_counter)
                values.append(number - r_counter)        
                nucludid.append(strText[3][i])
        
                index.append(number - r_counter)
                values.append(f_counter)             
                nucludid.append(strText[5][i])
                f_counter += 1        
                r_counter += 1                  
                continue        
            else:
                if(isChar(strText[4][i])):
                    index.append(f_counter)
                    values.append(0)        
                    nucludid.append(strText[3][i])
        
                    index.append(f_counter + 1)
                    values.append(0)        
                    nucludid.append(strText[4][i])
                
                    index.append(number - r_counter)
                    values.append(0)             
                    nucludid.append(strText[5][i])
                    f_counter += 2        
                    r_counter += 1                  
                    continue 
                else:
                    index.append(f_counter)
                    values.append(0)        
                    nucludid.append(strText[3][i])                                 
                    index.append(number - r_counter)
                    values.append(0)             
                    nucludid.append(strText[5][i])
                    f_counter += 1        
                    r_counter += 1                  
                    continue 
    # sort indexs and other base on indexes
    inds = np.array(index).argsort()
    index = np.array(index)[inds]
    values = np.array(values)[inds]
    nucludid = np.array(nucludid)[inds]
    # body
    text = ""    
    for i in range(len(index)):
        text += f"{adjust(index[i],6)} {nucludid[i]} {adjust(i,6)} {adjust((i+2)%(number+1),6)} {adjust(values[i],6)} {adjust(index[i],7)}\n"
    # common info
    deltaG = strText[0].split('(')[1].split(')')[0]    
    accession = strText[0].split(' ')[0][1:]
    # calc 5p
    has_5p = False
    s_5p = 0
    f_5p = 0
    counter = 0
    for i in range(len(strText[2])):
        if(isChar(strText[2][i]) or isChar(strText[3][i])):
            counter += 1            
            if(isMature(strText[2][i] ,strText[3][i])):
                if(not has_5p):
                    has_5p = True
                    s_5p = counter
                else:
                    continue
            elif(has_5p):
                f_5p = counter - 1 
                break                        
    # calc 3p
    has_3p = False
    s_3p = 0
    f_3p = 0
    counter = 0
    for i in range(len(strText[6])):
        if(isChar(strText[6][i]) or isChar(strText[5][i])):
            counter += 1            
            if(isMature(strText[6][i] ,strText[5][i])):
                if(not has_3p):
                    has_3p = True
                    s_3p = counter
                else:
                    continue
            elif(has_3p):
                f_3p = counter - 1 
                break        
    _s_3p = number - f_3p + 1
    _f_3p = number - s_3p + 1
    s_3p = _s_3p
    f_3p = _f_3p
    
    #             
    if(has_3p and has_5p):
        h5 = f"{adjust(number,6)} dG ={adjust(deltaG,10)} {accession}-5p|+|1-{number}|{s_5p}-{f_5p}\n"   
        h3 = f"{adjust(number,6)} dG ={adjust(deltaG,10)} {accession}-3p|+|1-{number}|{s_3p}-{f_3p}\n"        
        with open(f"CT/{accession}-5p|+|1-{number}|{s_5p}-{f_5p}.ct", 'w') as file:
            file.write(h5 + text)
        with open(f"CT/{accession}-3p|+|1-{number}|{s_3p}-{f_3p}.ct", 'w') as file:
            file.write(h3 + text)
    elif(has_5p):
        h5 = f"{adjust(number,6)} dG ={adjust(deltaG,10)} {accession}|+|1-{number}|{s_5p}-{f_5p}\n"   
        with open(f"CT/{accession}|+|1-{number}|{s_5p}-{f_5p}.ct", 'w') as file:
            file.write(h5 + text)
    elif(has_3p):
        h3 = f"{adjust(number,6)} dG ={adjust(deltaG,10)} {accession}|+|1-{number}|{s_3p}-{f_3p}\n"   
        with open(f"CT/{accession}|+|1-{number}|{s_3p}-{f_3p}.ct", 'w') as file:
            file.write(h3 + text)            

In [ ]:
with open('./Data/miRNA.str','r') as file:
    text = file.read()
    lines = text.split('\n')

In [127]:
i = 16
strText = lines[i:(i+7)]
print(strText[0])
print(strText[2])
print(strText[3])
print(strText[4])
print(strText[5])
print(strText[6])

>cel-mir-1 (-41.40)   [cel-miR-1-5p:21-42] [cel-miR-1-3p:61-81]
aaagug    ua   ag           C      GC     -    au 
      accg  ccg  cugCAUACUUC UUACAU  CCAUA cuau  c
      ||||  |||  ||||||||||| ||||||  ||||| ||||  a
      uggu  ggc  gAUGUAUGAAG AAUGUA  GGUau ggua  u
---uga    gg   aa           A      -A     a    aa 


In [128]:
convertor(strText)

     1 a      0      2      0       1
     2 a      1      3      0       2
     3 a      2      4      0       3
     4 g      3      5      0       4
     5 u      4      6      0       5
     6 g      5      7      0       6
     7 a      6      8     93       7
     8 c      7      9     92       8
     9 c      8     10     91       9
    10 g      9     11     90      10
    11 u     10     12      0      11
    12 a     11     13      0      12
    13 c     12     14     87      13
    14 c     13     15     86      14
    15 g     14     16     85      15
    16 a     15     17      0      16
    17 g     16     18      0      17
    18 c     17     19     82      18
    19 u     18     20     81      19
    20 g     19     21     80      20
    21 C     20     22     79      21
    22 A     21     23     78      22
    23 U     22     24     77      23
    24 A     23     25     76      24
    25 C     24     26     75      25
    26 U     25     27     74      26
    27 U    

In [106]:
for i in tqdm(range(0,len(lines),8)):
    strText = lines[i:(i+7)]       
    if(len(strText) == 7):
        convertor(strText)
    else:
        print(strText, "No 7 lines")

100%|██████████| 38590/38590 [00:57<00:00, 666.61it/s] 

[''] No 7 lines


In [107]:
!zip -r ct.zip ./CT

  adding: CT/ (stored 0%)
  adding: CT/mdo-mir-9a-3-3p|+|1-62|40-61.ct (deflated 72%)
  adding: CT/tca-mir-11649|+|1-61|38-60.ct (deflated 73%)
  adding: CT/pal-mir-1388-3p|+|1-60|38-60.ct (deflated 73%)
  adding: CT/ata-MIR160b-3p|+|1-107|79-99.ct (deflated 73%)
  adding: CT/nle-mir-98|+|1-80|1-22.ct (deflated 73%)
  adding: CT/ssc-mir-6782|+|1-75|44-65.ct (deflated 73%)
  adding: CT/hsa-mir-421|+|1-85|48-70.ct (deflated 73%)
  adding: CT/tca-mir-11-3p|+|1-106|63-84.ct (deflated 73%)
  adding: CT/gma-MIR10186g|+|1-128|95-118.ct (deflated 72%)
  adding: CT/prd-mir-7931-3p|+|1-64|43-64.ct (deflated 72%)
  adding: CT/bta-mir-671|+|1-118|29-51.ct (deflated 72%)
  adding: CT/dre-mir-129-4-5p|+|1-89|14-35.ct (deflated 73%)
  adding: CT/sha-mir-221|+|1-150|101-121.ct (deflated 74%)
  adding: CT/tca-mir-3805a-3p|+|1-96|59-81.ct (deflated 73%)
  adding: CT/cas-MIR169d|+|1-88|64-84.ct (deflated 73%)
  adding: CT/ipu-mir-9-1|+|1-62|1-23.ct (deflated 73%)
  adding: CT/vca-MIR397-3p|+|1-78|57-77.c

  adding: CT/ptc-MIR6462c-5p|+|1-133|16-37.ct (deflated 73%)
  adding: CT/bta-mir-423-5p|+|1-94|17-39.ct (deflated 73%)
  adding: CT/dno-mir-33b-5p|+|1-56|1-21.ct (deflated 72%)
  adding: CT/mdm-MIR398a|+|1-137|94-114.ct (deflated 73%)
  adding: CT/oni-mir-10690|+|1-61|1-23.ct (deflated 72%)
  adding: CT/ame-mir-6041|+|1-71|40-61.ct (deflated 73%)
  adding: CT/mmu-mir-7683-3p|+|1-61|41-61.ct (deflated 72%)
  adding: CT/ssa-mir-129-1-3p|+|1-65|44-65.ct (deflated 73%)
  adding: CT/mtr-MIR2111f|+|1-116|12-32.ct (deflated 73%)
  adding: CT/prd-mir-252-3p|+|1-62|42-62.ct (deflated 73%)
  adding: CT/ssa-mir-430a-3p|+|1-60|39-60.ct (deflated 72%)
  adding: CT/pab-MIR950b|+|1-112|25-46.ct (deflated 72%)
  adding: CT/cpi-mir-129-2-5p|+|1-88|14-34.ct (deflated 73%)
  adding: CT/cin-mir-4002-5p|+|1-53|1-20.ct (deflated 72%)
  adding: CT/mmu-mir-125a-5p|+|1-68|6-29.ct (deflated 73%)
  adding: CT/cfa-mir-2483|+|1-139|41-62.ct (deflated 72%)
  adding: CT/mmu-mir-204-3p|+|1-68|45-65.ct (deflated 73%)

  adding: CT/mmu-mir-465a-5p|+|1-74|5-27.ct (deflated 73%)
  adding: CT/mse-mir-9571a|+|1-103|69-89.ct (deflated 73%)
  adding: CT/bdi-MIR7716-3p|+|1-358|325-348.ct (deflated 72%)
  adding: CT/gga-mir-219a|+|1-85|10-30.ct (deflated 73%)
  adding: CT/bta-mir-199b|+|1-100|16-38.ct (deflated 73%)
  adding: CT/rlcv-mir-rL1-2-5p|+|1-79|13-34.ct (deflated 72%)
  adding: CT/ssa-mir-10d-1-5p|+|1-59|1-22.ct (deflated 73%)
  adding: CT/pma-mir-4543|+|1-74|8-29.ct (deflated 73%)
  adding: CT/gso-MIR1507b|+|1-98|72-93.ct (deflated 72%)
  adding: CT/bmo-mir-2812|+|1-89|60-79.ct (deflated 73%)
  adding: CT/sly-MIR10541|+|1-136|6-29.ct (deflated 73%)
  adding: CT/osa-MIR530-5p|+|1-157|11-30.ct (deflated 74%)
  adding: CT/mml-mir-762-1|+|1-71|48-69.ct (deflated 72%)
  adding: CT/ppe-MIR6288c-3p|+|1-88|57-80.ct (deflated 73%)
  adding: CT/efu-mir-9223b-2|+|1-80|11-31.ct (deflated 73%)
  adding: CT/lja-MIR11096b|+|1-52|32-52.ct (deflated 72%)
  adding: CT/cfa-mir-8845|+|1-125|36-55.ct (deflated 75%)
  a

  adding: CT/ssc-mir-222|+|1-80|51-74.ct (deflated 73%)
  adding: CT/eca-mir-323-5p|+|1-86|16-37.ct (deflated 73%)
  adding: CT/pbv-let-7i-5p|+|1-91|9-30.ct (deflated 73%)
  adding: CT/efu-mir-30a|+|1-85|52-75.ct (deflated 72%)
  adding: CT/mml-mir-193a-3p|+|1-85|52-73.ct (deflated 73%)
  adding: CT/mtr-MIR5289a|+|1-103|70-93.ct (deflated 73%)
  adding: CT/sbo-mir-199-3|+|1-60|40-60.ct (deflated 72%)
  adding: CT/dps-mir-2559-5p|+|1-109|18-42.ct (deflated 73%)
  adding: CT/nta-MIR6145f|+|1-81|11-31.ct (deflated 73%)
  adding: CT/hsa-mir-5590-5p|+|1-54|1-21.ct (deflated 72%)
  adding: CT/spu-mir-153-3p|+|1-101|61-82.ct (deflated 73%)
  adding: CT/mtr-MIR2629g|+|1-274|236-256.ct (deflated 74%)
  adding: CT/isc-mir-96|+|1-101|21-42.ct (deflated 73%)
  adding: CT/cja-mir-128-2|+|1-78|49-69.ct (deflated 73%)
  adding: CT/asu-mir-5355-3p|+|1-81|48-70.ct (deflated 73%)
  adding: CT/crm-mir-52-2-5p|+|1-93|18-41.ct (deflated 73%)
  adding: CT/gga-mir-6608-2-3p|+|1-110|63-85.ct (deflated 73%)
  

  adding: CT/stu-MIR390-3p|+|1-70|50-70.ct (deflated 73%)
  adding: CT/cgr-mir-1b|+|1-122|80-102.ct (deflated 75%)
  adding: CT/tae-MIR1131|+|1-100|4-25.ct (deflated 74%)
  adding: CT/asu-mir-250-3p|+|1-79|48-70.ct (deflated 72%)
  adding: CT/age-mir-513e-1|+|1-87|16-36.ct (deflated 73%)
  adding: CT/ppe-MIR6267c-3p|+|1-133|88-108.ct (deflated 72%)
  adding: CT/dno-mir-455-3p|+|1-58|37-58.ct (deflated 72%)
  adding: CT/oan-mir-204-3p|+|1-119|69-88.ct (deflated 73%)
  adding: CT/mdo-mir-7347-5p|+|1-65|1-23.ct (deflated 72%)
  adding: CT/mdo-mir-7386a-1-3p|+|1-61|40-61.ct (deflated 72%)
  adding: CT/mmu-mir-7688-3p|+|1-56|36-56.ct (deflated 72%)
  adding: CT/cja-mir-557|+|1-119|94-115.ct (deflated 73%)
  adding: CT/stu-MIR7999-5p|+|1-50|1-0.ct (deflated 72%)
  adding: CT/egr-mir-4989-3p|+|1-80|58-79.ct (deflated 73%)
  adding: CT/stu-MIR319-5p|+|1-177|1-21.ct (deflated 72%)
  adding: CT/aca-mir-5448|+|1-89|17-38.ct (deflated 73%)
  adding: CT/osa-MIR5789|+|1-65|40-60.ct (deflated 72%)
  

...

  adding: CT/aly-MIR399g-5p|+|1-157|23-44.ct (deflated 74%)
  adding: CT/oan-mir-1421p-2-3p|+|1-124|77-98.ct (deflated 73%)
  adding: CT/zma-MIR171c-3p|+|1-118|79-99.ct (deflated 74%)
  adding: CT/ppc-mir-2262|+|1-113|31-53.ct (deflated 73%)
  adding: CT/hsa-mir-1304-5p|+|1-91|20-41.ct (deflated 73%)
  adding: CT/ppc-mir-8227-5p|+|1-54|1-21.ct (deflated 72%)
  adding: CT/zma-MIR394b-5p|+|1-122|21-40.ct (deflated 72%)
  adding: CT/gma-MIR166q|+|1-152|122-141.ct (deflated 74%)
  adding: CT/bdi-MIR396b-3p|+|1-189|138-158.ct (deflated 73%)
  adding: CT/tca-mir-3895-3p|+|1-90|64-82.ct (deflated 74%)
  adding: CT/rno-mir-208a-5p|+|1-83|15-36.ct (deflated 73%)
  adding: CT/gga-mir-1685-3p|+|1-95|60-81.ct (deflated 73%)
  adding: CT/bna-MIR395a|+|1-83|58-78.ct (deflated 73%)
  adding: CT/xla-mir-222-5p|+|1-60|1-22.ct (deflated 72%)
  adding: CT/stu-MIR8038a-3p|+|1-99|79-99.ct (deflated 73%)
  adding: CT/dma-mir-708|+|1-69|48-69.ct (deflated 73%)
  adding: CT/dme-mir-994-5p|+|1-91|12-33.ct (def

  adding: CT/mmu-mir-669a-1-5p|+|1-97|20-43.ct (deflated 73%)
  adding: CT/bhv5-mir-B12|+|1-78|11-30.ct (deflated 73%)
  adding: CT/dme-mir-4985-3p|+|1-101|65-86.ct (deflated 74%)
  adding: CT/hsa-mir-3199-1|+|1-88|10-32.ct (deflated 73%)
  adding: CT/dre-let-7d-1-3p|+|1-97|63-84.ct (deflated 73%)
  adding: CT/dsi-mir-959|+|1-92|57-78.ct (deflated 73%)
  adding: CT/dvi-mir-996-3p|+|1-98|62-81.ct (deflated 73%)
  adding: CT/tgu-mir-2987-3p|+|1-80|49-70.ct (deflated 73%)
  adding: CT/gga-mir-1633|+|1-95|20-42.ct (deflated 73%)
  adding: CT/ata-MIR2275c-5p|+|1-256|14-34.ct (deflated 73%)
  adding: CT/hsa-mir-550b-2-5p|+|1-97|16-37.ct (deflated 73%)
  adding: CT/ggo-mir-141|+|1-95|60-80.ct (deflated 73%)
  adding: CT/hsv2-miR-H10|+|1-66|43-60.ct (deflated 73%)
  adding: CT/osa-MIR5149|+|1-376|343-364.ct (deflated 74%)
  adding: CT/ggo-mir-675b|+|1-113|21-43.ct (deflated 74%)
  adding: CT/eca-mir-514|+|1-89|55-75.ct (deflated 73%)
  adding: CT/hvt-mir-H8|+|1-76|9-32.ct (deflated 73%)
  addi

  adding: CT/bta-mir-331-5p|+|1-90|25-42.ct (deflated 73%)
  adding: CT/ocu-mir-452-3p|+|1-61|40-61.ct (deflated 72%)
  adding: CT/dvi-mir-976-5p|+|1-93|21-41.ct (deflated 73%)
  adding: CT/gmo-mir-124-4|+|1-60|40-60.ct (deflated 73%)
  adding: CT/mmu-mir-93-3p|+|1-88|54-75.ct (deflated 73%)
  adding: CT/cpi-mir-181a-3-5p|+|1-89|15-37.ct (deflated 73%)
  adding: CT/ssc-mir-19b-1|+|1-80|52-74.ct (deflated 73%)
  adding: CT/bma-mir-100b|+|1-91|21-42.ct (deflated 74%)
  adding: CT/rno-mir-322-2-3p|+|1-95|61-80.ct (deflated 73%)
  adding: CT/spu-mir-4848a|+|1-68|12-34.ct (deflated 73%)
  adding: CT/pma-mir-4611|+|1-87|57-77.ct (deflated 73%)
  adding: CT/api-mir-3055-1|+|1-58|38-58.ct (deflated 72%)
  adding: CT/ppy-mir-23b|+|1-97|58-80.ct (deflated 73%)
  adding: CT/pma-mir-4613|+|1-89|53-75.ct (deflated 73%)
  adding: CT/ggo-mir-483|+|1-112|21-42.ct (deflated 74%)
  adding: CT/rno-mir-93-5p|+|1-87|15-37.ct (deflated 73%)
  adding: CT/bmo-mir-11-3p|+|1-76|45-66.ct (deflated 73%)
  adding:

  adding: CT/lja-MIR5287c|+|1-156|126-149.ct (deflated 71%)
  adding: CT/mdo-mir-12331-3p|+|1-62|41-62.ct (deflated 73%)
  adding: CT/cli-mir-142-5p|+|1-92|18-38.ct (deflated 72%)
  adding: CT/cfa-mir-653|+|1-91|12-32.ct (deflated 73%)
  adding: CT/oni-mir-10613|+|1-56|1-23.ct (deflated 73%)
  adding: CT/mmu-mir-6769b-5p|+|1-60|5-24.ct (deflated 73%)
  adding: CT/cfa-mir-340|+|1-92|16-37.ct (deflated 73%)
  adding: CT/gga-mir-1557|+|1-70|40-59.ct (deflated 72%)
  adding: CT/ppt-MIR536c|+|1-222|151-172.ct (deflated 73%)
  adding: CT/hsa-mir-505-5p|+|1-84|15-36.ct (deflated 73%)
  adding: CT/ssc-mir-9828-3|+|1-98|68-88.ct (deflated 74%)
  adding: CT/pxy-mir-8510a-6-5p|+|1-77|14-34.ct (deflated 72%)
  adding: CT/ata-MIR395b-5p|+|1-214|11-31.ct (deflated 75%)
  adding: CT/nve-mir-2024a-2-5p|+|1-63|7-28.ct (deflated 72%)
  adding: CT/tgu-mir-138-1-5p|+|1-73|11-33.ct (deflated 73%)
  adding: CT/bmo-mir-184-3p|+|1-84|50-73.ct (deflated 73%)
  adding: CT/mmu-mir-142a-5p|+|1-64|4-24.ct (deflate

  adding: CT/hsa-mir-601|+|1-79|16-37.ct (deflated 73%)
  adding: CT/bta-mir-9851|+|1-56|36-56.ct (deflated 72%)
  adding: CT/sme-mir-124c-2-3p|+|1-88|58-78.ct (deflated 73%)
  adding: CT/dvi-mir-9544b-3p|+|1-61|39-59.ct (deflated 72%)
  adding: CT/bra-MIR390-3p|+|1-105|68-88.ct (deflated 73%)
  adding: CT/cpi-mir-29b-1-3p|+|1-91|54-74.ct (deflated 73%)
  adding: CT/gmo-mir-10c-5p|+|1-65|1-22.ct (deflated 73%)
  adding: CT/sko-mir-7-5p|+|1-86|16-38.ct (deflated 73%)
  adding: CT/sme-mir-2147d-5p|+|1-97|21-42.ct (deflated 73%)
  adding: CT/asu-mir-87b-5p|+|1-83|12-34.ct (deflated 73%)
  adding: CT/dmo-mir-9c|+|1-89|13-34.ct (deflated 73%)
  adding: CT/isc-mir-5310|+|1-110|65-85.ct (deflated 73%)
  adding: CT/gga-mir-6628|+|1-110|70-90.ct (deflated 73%)
  adding: CT/osa-MIR395a|+|1-142|112-132.ct (deflated 72%)
  adding: CT/hsa-mir-5194|+|1-120|73-94.ct (deflated 73%)
  adding: CT/prd-mir-7968-3p|+|1-62|41-62.ct (deflated 73%)
  adding: CT/asu-mir-100a-1-5p|+|1-77|8-31.ct (deflated 73%)
